In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1134781,2021-03-30,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1134782,2021-03-31,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1134783,2021-04-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1134784,2021-04-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
30511,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30513,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30515,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30517,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30519,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
700274,2021-03-30,Arkansas,Arkansas,5001,2076,36.00,5000,Arkansas,AR,Arkansas,State,3017804
700276,2021-03-31,Arkansas,Arkansas,5001,2076,36.00,5000,Arkansas,AR,Arkansas,State,3017804
700278,2021-04-01,Arkansas,Arkansas,5001,2077,36.00,5000,Arkansas,AR,Arkansas,State,3017804
700280,2021-04-02,Arkansas,Arkansas,5001,2077,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1134781,2021-03-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1134782,2021-03-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1134783,2021-04-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1134784,2021-04-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-03') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
438,2021-04-03,Snohomish,Washington,32600,558.00,53061,WA,County,822083,3965.54,67.88
874,2021-04-03,Cook,Illinois,502207,10248.00,17031,IL,County,5150233,9751.15,198.98
1309,2021-04-03,Orange,California,266766,4768.00,6059,CA,County,3175692,8400.25,150.14
1743,2021-04-03,Maricopa,Arizona,525361,9663.00,4013,AZ,County,4485414,11712.65,215.43
2177,2021-04-03,Los Angeles,California,1222262,23274.00,6037,CA,County,10039107,12175.01,231.83
...,...,...,...,...,...,...,...,...,...,...,...
1134217,2021-04-03,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1134390,2021-04-03,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1134532,2021-04-03,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1134670,2021-04-03,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
438,2021-04-03,Snohomish,Washington,32600,558.00,53061,WA,County,822083,3965.54,67.88,67.88,3965.54
874,2021-04-03,Cook,Illinois,502207,10248.00,17031,IL,County,5150233,9751.15,198.98,198.98,9751.15
1309,2021-04-03,Orange,California,266766,4768.00,6059,CA,County,3175692,8400.25,150.14,150.14,8400.25
1743,2021-04-03,Maricopa,Arizona,525361,9663.00,4013,AZ,County,4485414,11712.65,215.43,215.43,11712.65
2177,2021-04-03,Los Angeles,California,1222262,23274.00,6037,CA,County,10039107,12175.01,231.83,231.83,12175.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1134217,2021-04-03,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1134390,2021-04-03,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1134532,2021-04-03,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1134670,2021-04-03,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
732185,2021-04-03,Hale,Alabama,2170,73.00,1065,AL,County,14651,14811.28,498.26,498.26,14811.28,1
796855,2021-04-03,Clarke,Alabama,3453,60.00,1025,AL,County,23622,14617.73,254.00,254.00,14617.73,2
573081,2021-04-03,Lowndes,Alabama,1363,53.00,1085,AL,County,9726,14013.98,544.93,544.93,14013.98,3
426546,2021-04-03,Franklin,Alabama,4189,82.00,1059,AL,County,31362,13356.93,261.46,261.46,13356.93,4
508212,2021-04-03,Etowah,Alabama,13651,345.00,1055,AL,County,102268,13348.26,337.35,337.35,13348.26,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699169,2021-04-03,Sublette,Wyoming,710,7.00,56035,WY,County,9831,7222.05,71.20,71.20,7222.05,19
698797,2021-04-03,Converse,Wyoming,983,17.00,56009,WY,County,13822,7111.85,122.99,122.99,7111.85,20
868422,2021-04-03,Lincoln,Wyoming,1357,12.00,56023,WY,County,19830,6843.17,60.51,60.51,6843.17,21
966248,2021-04-03,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
573081,2021-04-03,Lowndes,Alabama,1363,53.00,1085,AL,County,9726,14013.98,544.93,544.93,14013.98,3,1
732185,2021-04-03,Hale,Alabama,2170,73.00,1065,AL,County,14651,14811.28,498.26,498.26,14811.28,1,2
243967,2021-04-03,Walker,Alabama,7069,274.00,1127,AL,County,63521,11128.60,431.35,431.35,11128.60,28,3
617958,2021-04-03,Greene,Alabama,906,34.00,1063,AL,County,8111,11170.02,419.18,419.18,11170.02,26,4
572333,2021-04-03,Crenshaw,Alabama,1501,57.00,1041,AL,County,13772,10898.93,413.88,413.88,10898.93,30,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616465,2021-04-03,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
868422,2021-04-03,Lincoln,Wyoming,1357,12.00,56023,WY,County,19830,6843.17,60.51,60.51,6843.17,21,20
846828,2021-04-03,Uinta,Wyoming,2139,12.00,56041,WY,County,20226,10575.50,59.33,59.33,10575.50,4,21
242824,2021-04-03,Teton,Wyoming,3624,9.00,56039,WY,County,23464,15444.94,38.36,38.36,15444.94,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,17,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,17,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,17,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,17,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,17,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132114,2021-03-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1132115,2021-03-31,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1132116,2021-04-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1132117,2021-04-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
729149,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14811.28,2,1,1.00
729150,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14811.28,2,1,0.00
729151,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14811.28,2,1,0.00
729152,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14811.28,2,1,0.00
729153,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,498.26,14811.28,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882906,2021-03-30,Crook,Wyoming,424,11.00,56011,WY,County,7584,5590.72,145.04,158.23,5590.72,6,23,0.00
882907,2021-03-31,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
882908,2021-04-01,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
882909,2021-04-02,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
570785,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,1.00,0.00
570786,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
570787,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
570788,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
570789,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14013.98,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568910,2021-03-30,Albany,Wyoming,3969,11.00,56001,WY,County,38880,10208.33,28.29,28.29,10246.91,23,6,1.00,0.00
568911,2021-03-31,Albany,Wyoming,3970,11.00,56001,WY,County,38880,10210.91,28.29,28.29,10246.91,23,6,1.00,0.00
568912,2021-04-01,Albany,Wyoming,3979,11.00,56001,WY,County,38880,10234.05,28.29,28.29,10246.91,23,6,9.00,0.00
568913,2021-04-02,Albany,Wyoming,3984,11.00,56001,WY,County,38880,10246.91,28.29,28.29,10246.91,23,6,5.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-03') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
291179,2021-04-03,Imperial,California,27553,711.00,6025,CA,County,181215,15204.59,392.35,392.35,15204.59,1,2,33.00,0.00
2177,2021-04-03,Los Angeles,California,1222262,23274.00,6037,CA,County,10039107,12175.01,231.83,231.83,12175.01,2,5,756.00,39.00
145095,2021-04-03,San Bernardino,California,291360,4417.00,6071,CA,County,2180085,13364.62,202.61,202.61,13364.62,3,4,161.00,13.00
509664,2021-04-03,Inyo,California,1377,36.00,6027,CA,County,18039,7633.46,199.57,199.57,7633.46,4,25,3.00,0.00
64933,2021-04-03,Stanislaus,California,59342,1003.00,6099,CA,County,550660,10776.52,182.15,182.15,10776.52,5,9,71.00,0.00
30117,2021-04-03,Riverside,California,294736,4362.00,6065,CA,County,2470546,11929.99,176.56,176.56,11929.99,6,6,0.00,0.00
80490,2021-04-03,Tulare,California,49100,818.00,6107,CA,County,466195,10532.07,175.46,175.46,10532.07,7,10,0.00,1.00
51675,2021-04-03,San Joaquin,California,70117,1310.00,6077,CA,County,762148,9199.92,171.88,171.88,9199.92,8,19,0.00,0.00
29331,2021-04-03,Fresno,California,99551,1620.00,6019,CA,County,999101,9964.06,162.15,162.15,9964.06,9,14,234.00,4.00
461929,2021-04-03,Merced,California,30687,439.00,6047,CA,County,277680,11051.21,158.10,158.10,11051.21,10,8,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1076632,2021-04-03,Lassen,California,5652,1.00,6035,CA,County,30573,18486.90,3.27,3.27,18486.90,56,1,0.00,0.00
291179,2021-04-03,Imperial,California,27553,711.00,6025,CA,County,181215,15204.59,392.35,392.35,15204.59,1,2,33.00,0.00
617900,2021-04-03,Kings,California,22666,241.00,6031,CA,County,152940,14820.19,157.58,157.58,14820.19,11,3,7.00,0.00
145095,2021-04-03,San Bernardino,California,291360,4417.00,6071,CA,County,2180085,13364.62,202.61,202.61,13364.62,3,4,161.00,13.00
2177,2021-04-03,Los Angeles,California,1222262,23274.00,6037,CA,County,10039107,12175.01,231.83,231.83,12175.01,2,5,756.00,39.00
30117,2021-04-03,Riverside,California,294736,4362.00,6065,CA,County,2470546,11929.99,176.56,176.56,11929.99,6,6,0.00,0.00
182385,2021-04-03,Kern,California,106420,1281.00,6029,CA,County,900202,11821.79,142.30,142.30,11821.79,14,7,91.00,4.00
461929,2021-04-03,Merced,California,30687,439.00,6047,CA,County,277680,11051.21,158.10,158.10,11051.21,10,8,0.00,0.00
64933,2021-04-03,Stanislaus,California,59342,1003.00,6099,CA,County,550660,10776.52,182.15,182.15,10776.52,5,9,71.00,0.00
80490,2021-04-03,Tulare,California,49100,818.00,6107,CA,County,466195,10532.07,175.46,175.46,10532.07,7,10,0.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
291179,2021-04-03,Imperial,California,27553,711.00,6025,CA,County,181215,15204.59,392.35,392.35,15204.59,1,2,33.00,0.00
2177,2021-04-03,Los Angeles,California,1222262,23274.00,6037,CA,County,10039107,12175.01,231.83,231.83,12175.01,2,5,756.00,39.00
145095,2021-04-03,San Bernardino,California,291360,4417.00,6071,CA,County,2180085,13364.62,202.61,202.61,13364.62,3,4,161.00,13.00
509664,2021-04-03,Inyo,California,1377,36.00,6027,CA,County,18039,7633.46,199.57,199.57,7633.46,4,25,3.00,0.00
64933,2021-04-03,Stanislaus,California,59342,1003.00,6099,CA,County,550660,10776.52,182.15,182.15,10776.52,5,9,71.00,0.00
30117,2021-04-03,Riverside,California,294736,4362.00,6065,CA,County,2470546,11929.99,176.56,176.56,11929.99,6,6,0.00,0.00
80490,2021-04-03,Tulare,California,49100,818.00,6107,CA,County,466195,10532.07,175.46,175.46,10532.07,7,10,0.00,1.00
51675,2021-04-03,San Joaquin,California,70117,1310.00,6077,CA,County,762148,9199.92,171.88,171.88,9199.92,8,19,0.00,0.00
29331,2021-04-03,Fresno,California,99551,1620.00,6019,CA,County,999101,9964.06,162.15,162.15,9964.06,9,14,234.00,4.00
461929,2021-04-03,Merced,California,30687,439.00,6047,CA,County,277680,11051.21,158.10,158.10,11051.21,10,8,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15204.59,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15204.59,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15204.59,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15204.59,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15204.59,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3812,03/30/21,Lassen,5652,1.00,18486.90,3.27,3.27,18486.90,56,1,-1.00,0.00
3813,03/31/21,Lassen,5652,1.00,18486.90,3.27,3.27,18486.90,56,1,0.00,0.00
3814,04/01/21,Lassen,5652,1.00,18486.90,3.27,3.27,18486.90,56,1,0.00,0.00
3815,04/02/21,Lassen,5652,1.00,18486.90,3.27,3.27,18486.90,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15204.59,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15204.59,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15204.59,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15204.59,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15204.59,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3898,03/30/21,Merced,30558,435.00,11004.75,156.66,158.10,11051.21,10,8,37.00,0.00
3899,03/31/21,Merced,30593,436.00,11017.36,157.02,158.10,11051.21,10,8,35.00,1.00
3900,04/01/21,Merced,30649,438.00,11037.53,157.74,158.10,11051.21,10,8,56.00,2.00
3901,04/02/21,Merced,30687,439.00,11051.21,158.10,158.10,11051.21,10,8,38.00,1.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)